In [1]:
# using Pkg
# Pkg.activate("/home/jek354/research/ML-signproblem")
# Pkg.update()

using Lattices
using LinearAlgebra
using Combinatorics
using SparseArrays
using Plots
import Graphs
using LaTeXStrings
using Statistics
using Random
using Zygote
using Optimization, OptimizationOptimisers
using JSON
using OptimizationOptimJL
using JLD2
# using ExponentialUtilities


include("ed_objects.jl")
include("ed_functions.jl")
include("ed_optimization.jl")
include("utility_functions.jl")
include("adiabatic_analysis.jl")

time_ordered_unitary

In [61]:
# using KrylovKit


t = 1.0
U = 6
μ = 0  # positive incentivises fewer particles (one electron costs this much energy)
N_up = 2
N_down = 2
N =  3
half_filling = false
# lattice = Chain(6, Periodic())
lattice_dimension = (2,3)
bc = "periodic"
# lattice = Chain(6, Periodic())
lattice = Square(lattice_dimension, if bc == "periodic" Periodic() else Open() end)
# lattice = Graphs.cycle_graph(3)

models = []

reference_index = 2
# t_values = []# Float64[1,1], Float64[1,0.1], 1.0]
# for _t in t_values
#     # println(_t)
#     push!(models,HubbardModel(_t,0.0001,μ,half_filling))
# end
# U_values = [0.00001,0.01]
U_values = [0.00001; LinRange(2.1,9,20)]
U_values = sort([U_values; 10.0 .^LinRange(-3,2,40)])

for U in U_values
    # println(t)
    push!(models,HubbardModel(t,U,μ,half_filling))
end

subspace = HubbardSubspace(N, lattice)
# subspace = HubbardSubspace(N_up, N_down, lattice)

symmetry_labels= []
symmetry_ops = []
if subspace.N >= 1
    push!(symmetry_ops,cis(π*Matrix(create_operator(subspace,:Sx))))
    push!(symmetry_labels, "exp(iπSx)")
    push!(symmetry_ops,create_operator(subspace,:Sx))
    push!(symmetry_labels, "Sx")
end
push!(symmetry_ops, create_operator(subspace,:S2))
push!(symmetry_labels, "S2")
# op3 = Matrix(create_operator(subspace,:L2))
push!(symmetry_ops, create_operator(subspace,:T, kind=1))
push!(symmetry_labels, "Tx")
push!(symmetry_ops, create_operator(subspace,:T, kind=2))
push!(symmetry_labels, "Ty")
# push!(symmetry_ops, create_operator(subspace,:σ, kind=1))
E = []
H = []
V = []
# for model ∈ models
#     push!(H, Matrix(create_Hubbard(model, subspace; perturbations=false)))
#     e, v = eigen(H[end])
#     push!(E, e)
#     push!(V, v)
# end

H, indexer = create_Hubbard(models[20], subspace; get_indexer=true)


(sparse([1, 2, 3, 5, 7, 12, 1, 2, 4, 9  …  212, 217, 219, 220, 209, 214, 216, 218, 219, 220], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2  …  219, 219, 219, 219, 220, 220, 220, 220, 220, 220], [0.0, -2.0, 1.0, 1.0, 1.0, -1.0, -2.0, 0.0, 1.0, -1.0  …  1.0, 1.0, 0.0, -2.0, 1.0, -1.0, 1.0, 1.0, -2.0, 0.0], 220, 220), CombinationIndexer{Coordinate{2, Int64}}(Coordinate{2, Int64}[Coordinate(1, 1), Coordinate(2, 1), Coordinate(1, 2), Coordinate(2, 2), Coordinate(1, 3), Coordinate(2, 3)], Dict{Tuple{Set{Coordinate{2, Int64}}, Set{Coordinate{2, Int64}}}, Int64}((Set([Coordinate(2, 3), Coordinate(2, 2)]), Set([Coordinate(1, 2)])) => 191, (Set([Coordinate(2, 1)]), Set([Coordinate(1, 3), Coordinate(2, 2)])) => 48, (Set([Coordinate(1, 2)]), Set([Coordinate(1, 3), Coordinate(1, 1)])) => 54, (Set([Coordinate(1, 2), Coordinate(2, 1)]), Set([Coordinate(2, 3)])) => 146, (Set(), Set([Coordinate(1, 1), Coordinate(2, 2), Coordinate(1, 2)])) => 5, (Set([Coordinate(2, 3)]), Set([Coordinate(1, 1), Coordinate(2, 2)])) => 9

In [ ]:
include("ed_functions.jl")
t_dict = create_randomized_nth_order_operator(2, indexer; omit_H_conj=false, conserve_spin=subspace.N < 0)
rows, cols, signs, ops_list = build_n_body_structure(t_dict, indexer)
param_index_map = build_param_index_map(ops_list, collect(keys(t_dict)))
use_symmetry = true
dim = length(indexer.inv_comb_dict)
t_vals = collect(values(t_dict))

# no symmetry 
if !use_symmetry
    ops = []
    for k in collect(keys(t_dict))
        rows, cols, signs, ops_list = build_n_body_structure(Dict(k=>1), indexer)
        push!(ops, make_hermitian(sparse(rows, cols, signs, dim, dim)))
    end

    # comparing implementations
    x1 = Matrix(sum(a*op for (a,op) in zip(t_vals, ops)))

    rows, cols, signs, ops_list = build_n_body_structure(t_dict, indexer)
    vals = update_values(signs, param_index_map, t_vals)
    x2 = make_hermitian(sparse(rows, cols, vals, dim, dim))

    display(sum(abs2,x1-x2))
else
    # with symmetry
    ops = []
    t_keys = collect(keys(t_dict))
    inv_param_map, parameter_mapping, parity = find_symmetry_groups(t_keys, maximum(indexer.a).coordinates..., 
        hermitian=true, trans_x=true, trans_y=true, spin_symmetry=true)
    
    reduced::Vector{ComplexF64} = rand(length(inv_param_map))
    # reduced .= 0
    # reduced[1] = 1

    for (i,key_idcs) in enumerate(inv_param_map)
        tmp_t_dict::Dict{Array{Tuple{Coordinate{2, Int64}, Int64, Symbol}, 1}, Float64} = Dict()
        for key_idx in key_idcs
            tmp_t_dict[t_keys[key_idx]] = parity[key_idx]
            # println(t_keys[key_idx])
            # println(parity[key_idx])
        end
        _rows, _cols, _signs, _ops_list = build_n_body_structure(tmp_t_dict, indexer)
        _param_index_map = build_param_index_map(_ops_list, collect(keys(tmp_t_dict)))
        _vals = update_values(_signs, _param_index_map, collect(values(tmp_t_dict)))
        push!(ops, sparse(_rows, _cols, _vals, dim, dim))
        # break
    end

    println("Number of free parameters: $(length(inv_param_map))")
    x1 = sum(a*op for (a,op) in zip(reduced, ops))
    for (label,sym_op) in zip(symmetry_labels,symmetry_ops)
        println("$label: $(sum(abs2,sym_op*x1 - x1*sym_op))")
    end

    println()
    vals = update_values(signs, param_index_map,reduced, parameter_mapping, parity)
    x2 = sparse(rows,cols, vals, dim, dim)
    for (label,sym_op) in zip(symmetry_labels,symmetry_ops)
        println("$label: $(sum(abs2,sym_op*x2 - x2*sym_op))")
    end
    println()
    println("Different method matrix difference: $(sum(abs2, x1-x2))")

    # display(x1)
    # display(x2)
end

# dim = length(indexer.inv_comb_dict)
# vals = update_values(signs, param_index_map,reduced, param_map, parity)
# op = Matrix(make_hermitian(sparse(rows,cols, vals, dim, dim)))


Number of free parameters: 197
exp(iπSx): 2.2251356907392262e-27
Sx: 6700.622252222267
S2: 47294.678497565685
Tx: 0.0
Ty: 0.0

exp(iπSx): 2.2256287288049893e-27
Sx: 6700.622252222267
S2: 47294.678497565685
Tx: 2.9582283945787943e-31
Ty: 2.9582283945787943e-31

Different method matrix difference: 2.465190328815662e-31


In [58]:
r,c,_ = findnz(symmetry_ops[1])
println(indexer.inv_comb_dict[r[1]])
println(indexer.inv_comb_dict[c[1]])


(Set(Coordinate{2, Int64}[Coordinate(1, 1)]), Set(Coordinate{2, Int64}[Coordinate(1, 2), Coordinate(2, 1)]))
(Set{Coordinate{2, Int64}}(), Set(Coordinate{2, Int64}[Coordinate(1, 2), Coordinate(1, 1), Coordinate(2, 1)]))
